In [6]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from topogram_client import TopogramAPIClient

from csv import DictReader
import os
import json
import pymongo
#import musicbrainzngs
import arrow
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict
from dateutil import parser
from datetime import timedelta
import pickle
import time
from slugify import slugify
from fuzzywuzzy import fuzz
import urllib
import math

##SET PATH TO SAVE FILES:
savpath="./bands_dicts_GZ/"


In [7]:
#a function to have a range of colors for nodes/edges 
import colorsys
def get_N_HexCol(N=20):

    HSV_tuples = [(x*1.0/N, 0.5, 0.5) for x in xrange(N)]
    hex_out = []
    for rgb in HSV_tuples:
        rgb = map(lambda x: int(x*255),colorsys.hsv_to_rgb(*rgb))
        hex_out.append("".join(map(lambda x: chr(x).encode('hex'),rgb)))
    return hex_out

In [29]:
def create_topogram(title, nodes, edges):

    print "Creating topogram '%s'"%title

    try :
        r = topogram.create_topogram(title)
    except ValueError:
        print '> Topogram already exists'
        r = topogram.get_topogram_by_name(title)
    print r
    topogram_ID = r["data"]["_id"]
    print "topogram ID : %s"%topogram_ID

    # get and backup existing nodes and edges
    existing_nodes = topogram.get_nodes(topogram_ID)["data"]
    existing_edges = topogram.get_edges(topogram_ID)["data"]

    # clear existing graph
    if len(existing_nodes):
        topogram.delete_nodes([n["_id"] for n in existing_nodes])
        print "%s nodes deleted"%len(existing_nodes)
    if len(existing_edges):
        topogram.delete_edges([n["_id"] for n in existing_edges])
        print "%s edges deleted"%len(existing_edges)

    r = topogram.create_nodes(topogram_ID, nodes)
    #print r
    print "%s nodes created."%len(r["data"])
    r = topogram.create_edges(topogram_ID, edges)
    print "%s edges created."%len(r["data"])

    print "done. Topogram has been updated. Check it at %s/topograms/%s"%(TOPOGRAM_URL, topogram_ID)



In [3]:
artist={}
artist["fileID"]="david-guetta302bd7b9-d012-4360-897a-93b00c855680"
artist["mbid"]="a39acc4a-985e-4173-a9ed-f325f2d3bc1c"

with open("../../4_miner/tour_miner/"+artist["fileID"]+".obj", 'rb') as handle:
          tempL= pickle.load( handle)
print tempL

{1: {'tourLength': 7, 'tourdist': 8458.945541198262, 'tourYear': 2007, 'tourdistOpt': 9311.573373830164}, 2: {'tourLength': 3, 'tourdist': 9493.805761503612, 'tourYear': 2008, 'tourdistOpt': 9493.805761503612}, 3: {'tourLength': 8, 'tourdist': 8150.5474739437705, 'tourYear': 2008, 'tourdistOpt': 8988.507021383968}, 4: {'tourLength': 5, 'tourdist': 3254.90504466328, 'tourYear': 2008, 'tourdistOpt': 1777.3187851979756}, 5: {'tourLength': 6, 'tourdist': 6618.2781191442755, 'tourYear': 2009, 'tourdistOpt': 6337.4189518733765}, 6: {'tourLength': 8, 'tourdist': 17785.538656672405, 'tourYear': 2009, 'tourdistOpt': 14613.104319193675}, 7: {'tourLength': 22, 'tourdist': 45949.41120012457, 'tourYear': 2009, 'tourdistOpt': 31901.848489660268}, 8: {'tourLength': 30, 'tourdist': 62171.568171502804, 'tourYear': 2009, 'tourdistOpt': 39558.34712494696}, 9: {'tourLength': 21, 'tourdist': 85092.63300389866, 'tourYear': 2010, 'tourdistOpt': 46018.9862621119}, 10: {'tourLength': 4, 'tourdist': 3216.770394

In [4]:
###HERE WE BUILD A LABEL OWNERSHIP CRUSHER FACTORIZED
with open("../../4_miner/labels_aliases_TOTAL.obj", 'rb') as handle:
          tempRelAll= pickle.load( handle)
def find_owning_label(label):
    
    labrels=[]
    if "label-relation-list" in tempRelAll[label]["label"] :

        for relat in tempRelAll[label]["label"]["label-relation-list"]:
            if "direction" in relat and relat["direction"] == "backward":
                print tempRelAll[label]["label"]["name"],"OWNED BY",relat["target"]," ",tempRelAll[relat["target"]]["label"]["name"]
                labrels.append(relat["target"])
                rum_and_whisky=""
                while rum_and_whisky != "SELF": 
                    rum_and_whisky=find_owning_label(relat["target"])
                print "ENDED LOOP"
                labrels.remove(relat["target"])
                print "LABRELS",labrels
                #if labrels[0]==relat["target"]:
                 #   break
                
            else:
                pass
            
        
        if not labrels :
            print "TOP OF THE TREE FOR ",tempRelAll[label]["label"]["name"]
            return "SELF"
        else:
            return labrels
                


artist={}
artist["name"]="Steve Aoki"
artist["mbid"]="a39acc4a-985e-4173-a9ed-f325f2d3bc1c"

with open("../../4_miner/release_mbnz/"+artist["name"]+artist["mbid"]+".obj", 'rb') as handle:
          tempRelz= pickle.load( handle)
#print tempRel["release-list"][0]['label-info-list'][0]['label']['id']
#print tempRel["release-list"][0]['release-event-list'][0]["date"]




for release in tempRelz["release-list"]:
    print"---------------------------"
    print release['label-info-list'][0]['label']['id']
    print release['label-info-list'][0]['label']['name']
    print release['release-event-list'][0]["date"]
    
    #print tempRelAll[release['label-info-list'][0]['label']['id']]["label"]
    try:
        result_lab=find_owning_label(release['label-info-list'][0]['label']['id'])
        print "REZLAB",result_lab 
        print"XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
    except KeyError as e:
        #print e
        pass

EOFError: 

In [43]:
with open("../LiSTGZ.obj", 'rb') as handle:
          LISTGZ= pickle.load( handle)
for ART in LISTGZ:
    print ART


wolves-in-the-throne-room
woods
regal
hot-tears
melt-banana
dan-sartain
shining
scout-niblett
dalek
black-cobra
the-eye-of-time
salsa
ovo
geneva-jacuzzi
expo
lotus-fucker
jeffrey-lewis-los-bolts
wavves
sages-comme-des-sauvages
dj
trophy-wife
mike-watt-the-missingmen
sleepy-sun
chris-cohen
gable
damily
dysrhythmia
deerhoof
id
stuntman
monarch
tacocat
lithics
the-arson-project
musique
sourdure
acid-mother-temple
year-of-no-light
surrender
the-ex
tacos
institute
yonatan-gat
gun-outfit
france
robot-orchestra
frankie-cosmos
sun-araw
baseck
dirty-beaches
kurt
ducktails
unsane
container
cough
kylesa
ar-ker
the-space-lady
reverie
monochrome
salo
bob-log-iii
guerilla-toss
ghold
cobra
red-alert
part-chimp
guantanamo-baywatch
requin-chagrin
shopping
cancelled
bermuda
marisa-anderson
dan-friel
mdou-moctar
low
sete-star-sept
terry-malts
stagger-lee
maserati
marissa-nadler
dead-horses
helen-money
ed-schrader-s-music-beat
purling-hiss
cowtown
health
tune-yards
matchess
weasel-walter
your-old-droog
k

In [34]:
## setup mongo
client = MongoClient()
db = client["bandstour"]
bandsintown =  db["minedArtists"]

# credentials
TOPOGRAM_URL = "https://app.topogram.io"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"

# data
#title = "Foo Fighters bandstour 0.1"
my_nodesdict = {}
my_nodes= []
my_edges = []
my_edgesdict = {}
my_nodesdictGLOBAL={}

#LIVENATION CATALOGUE
liste_salles_LN = json.load(open('./JSON_LIVENATION_VENUE.json'))

liste_salle_parsee_LN={}
for salle in liste_salles_LN:
    if liste_salles_LN[salle]["type"]=="venue":
 
            liste_salle_parsee_LN[slugify(liste_salles_LN[salle]["title"])]={"lat":liste_salles_LN[salle]["position"]["lat"],"lng":liste_salles_LN[salle]["position"]["lng"]}
#print liste_salle_parsee_LN


# date parsing
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
DATETIME_FORMAT_FOR_MSBNZ ='%Y-%m-%d'

#retrieve trip coords for each artist
dict_coords={}
colors_for_nodes_and_edges = get_N_HexCol()
LIST=['david-guetta']
#for artist in db.minedArtists.find({"totalKm": {"$gte": 250000}}) :
for ART in LIST:
 print ART
 for artist in db.minedArtists.find({"slug" : ART}) :
  title = artist["name"]   
  if os.path.exists(savpath+slugify(artist["name"]+str(artist["mbid"]))+".obj")==True:
        my_nodesdict = {}
        my_nodesdictList=[]
        print "SKIPPED PROCESSING BUT TAKING INTO ACCOUNT ",artist["name"]
        with open(savpath+slugify(artist["name"]+str(artist["mbid"]))+".obj", 'rb') as handle:
          tempL= pickle.load( handle)
    #print 
        tit=tempL[0].encode("utf-8")
        #edgestop=temp[2]
        my_nodesdictList=tempL[1]
        for node in my_nodesdictList:
            my_nodesdict[node["data"]["id"]]=node

        #print my_nodes
        #pass
 else:
    my_nodesdict = {}
    my_nodes= []
    my_edges = []
    my_nodesdictList=[]
    my_edgesdict = {}
    #colors_for_nodes_and_edges = get_N_HexCol()
    
    print artist["name"]
    #print artist
    dates_coords = []
    dates = artist.get("gigs")
    tours = artist.get("tours")    
    singlegigs = artist.get("singleGigs")
    dateini=0
    touring=0
    tour_indice=0
    tour_counter_for_CO2=1
    tour_counter_for_CO2_lock=0
    tmpOLDgig ={}
    with open("../../4_miner/tour_miner/"+slugify(artist["name"]+str(artist["mbid"]))+".obj", 'rb') as handle:
                    tempCO2= pickle.load( handle)
    #print tempCO2
    #print tours
    for tour in tours:
        
        ##WE NEED TO MATCH TOUR INDICES OF CO2 SPECS FILES...
     if tour_counter_for_CO2_lock == 1:
        tour_counter_for_CO2+=1
     tour_counter_for_CO2_lock=0
     tour_color = "#"+colors_for_nodes_and_edges[tour_indice % 20]
     #print tour_color
     #print tour
     gig_indice =0
     dates = tour.get("gigs")
     for date in dates:
        #print date.get("datetime")
        venue = date.get("venue")
        latitude = venue.get("latitude")
        longitude = venue.get("longitude")
        #print venue
        ###concat key
        idd= slugify(venue["name"])+str(int(venue["latitude"]))+str(int(venue["longitude"]))
        
        if idd not in my_nodesdict:
            print "NOT", idd
            
            node = {
                    "id" : idd,
                    "name" : slugify(venue["name"]),
                    "lat" : float(venue["latitude"]),
                    "lng" : float(venue["longitude"]),
                    "weight" : float(1),
                    "start" : date.get("datetime").isoformat(),
                    "end" : (date.get("datetime")+timedelta(days=1)).isoformat(),
                    "color" : tour_color,
                    "notes" : "name : "+slugify(venue["name"])+"  \nlat : "+str(float(venue["latitude"]))+"  \nlng :"+str(float(venue["longitude"]))+"  \n weight :"+str(1)+"  \nstart : "+str(date.get("datetime").isoformat())+"  \nend : "+str((date.get("datetime")+timedelta(days=1)).isoformat())+"  \ncolor :"+str(tour_color)+"  \n  <a href=\"https://google.com/search?q="+slugify(venue["name"])+"\" target=\"_blank\"> "+slugify(venue["name"])+" </a>"
                #+"  \n[search venue named "+slugify(venue["name"])+" on google](https://google.com/search?q="+slugify(venue["name"])+")"
                    }
        else:
            
                print "YES",idd
                node = {
                    "id" : idd,
                    "name" : slugify(venue["name"]),
                    "lat" : float(venue["latitude"]),
                    "lng" : float(venue["longitude"]),
                    "weight" : math.sqrt(float(int(round(float(my_nodesdict[idd]["data"].get("weight"))**2))+1)) ,
                    "start" : my_nodesdict[idd]["data"].get("start"),
                    "end" : (date.get("datetime")+timedelta(days=1)).isoformat(),
                    "color" : tour_color,
                    "notes" : "name : "+slugify(venue["name"])+"  \nlat : "+str(float(venue["latitude"]))+"  \nlng :"+str(float(venue["longitude"]))+"  \n weight :"+str((float(my_nodesdict[idd]["data"].get("weight")))**2)+"  \nstart : "+str(my_nodesdict[idd]["data"].get("start"))+"  \nend : "+(date.get("datetime")+timedelta(days=1)).isoformat()+"  \ncolor :"+tour_color+"  \n  <a href=\"https://google.com/search?q="+slugify(venue["name"])+"\" target=\"_blank\"> "+slugify(venue["name"])+" </a>"
                    #+"  \n[search venue named "+slugify(venue["name"])+" on google](https://google.com/search?q="+slugify(venue["name"])+")"
                    
                    }
                print my_nodesdict[idd]["data"].get("weight")
                print float(my_nodesdict[idd]["data"].get("weight"))
               
                print float(my_nodesdict[idd]["data"].get("weight"))**2
                print int(float(my_nodesdict[idd]["data"].get("weight"))**2)
                print int(round(float(my_nodesdict[idd]["data"].get("weight"))**2))
                print float(int(float(my_nodesdict[idd]["data"].get("weight"))**2)+1)
                print math.sqrt(float(int(float(my_nodesdict[idd]["data"].get("weight"))**2)+1))
        if dateini == 0:
                dateini =1
        else:
                #print my_nodes[-1]["data"]["id"]
                #print dates[gig_indice-1]["distanceToNextGig"]
                
                
                
                notesE=""
                notesE+="distance : "
                try: 
                    notesE+=str(dates[gig_indice]["distanceToNextGig"])
                except KeyError:
                    notesE+="dates[gig_indice][distanceToNextGig] NOT AVAILABLE"
                notesE+=" km  \n"
                notesE+="source : "
                notesE+=str(my_nodes[-1]["data"]["id"])
                notesE+="  target : "
                notesE+=str(node["id"])
                notesE+="  \ndatesource : "
                notesE+=dates[gig_indice -1]["datetime"].isoformat()
                notesE+="  \ndatetarget : "
                notesE+=dates[gig_indice]["datetime"].isoformat()
                notesE+="  \ngroup: "
                notesE+=str(int(tour_indice)+1)
                #notesE+=
                
                
                #print gig_indice
                edge = {
                    "source" : my_nodes[-1]["data"]["id"],
                    "target" : node["id"],
                    "color" :  tour_color,
                    "notes" : notesE,
                    #"weight" : float(e["weight"]),
                    "start" : dates[gig_indice -1]["datetime"].isoformat(),
                    "end" : dates[gig_indice]["datetime"].isoformat(),
                    "group" : str(tour_counter_for_CO2)
                    
                }
                #print edge
                #ADDING EDGES GROUPS/CO2 HERE
                
                if tour["nbGigs"] > 2:
                    try:
                        if tempCO2[tour_counter_for_CO2]['tourdist'] > 1.25*tempCO2[tour_counter_for_CO2]['tourdistOpt']:
                            
                            edge["group"]+=" DASHED2"
                            edge["notes"]+="  \n tournée grandement optimisable  \nDistance parcourue pendant le tour: "+str(tempCO2[tour_counter_for_CO2]['tourdist'])+" km  \n"
                            edge["notes"]+="Distance recalculée: "+str(tempCO2[tour_counter_for_CO2]['tourdistOpt'])+" km  \n"
                            try:
                                edge["notes"]+="Taux d'optimisation "+str((tempCO2[tour_counter_for_CO2]['tourdist']-tempCO2[tour_counter_for_CO2]['tourdistOpt'])/tempCO2[tour_counter_for_CO2]['tourdist'])+" %"
                            except:
                                print "OUPS 2"
                        elif (tempCO2[tour_counter_for_CO2]['tourdist'] < 1.25*tempCO2[tour_counter_for_CO2]['tourdistOpt']) and (tempCO2[tour_counter_for_CO2]['tourdist'] > 1.1*tempCO2[tour_counter_for_CO2]['tourdistOpt']):
                            
                            edge["group"]+=" DASHED1"
                            edge["notes"]+="  \n  tournée optimisable  \nDistance parcourue pendant le tour: "+str(tempCO2[tour_counter_for_CO2]['tourdist'])+" km  \n"
                            edge["notes"]+="Distance recalculée: "+str(tempCO2[tour_counter_for_CO2]['tourdistOpt'])+" km  \n"
                            try:
                                edge["notes"]+="Taux d'optimisation "+str((tempCO2[tour_counter_for_CO2]['tourdist']-tempCO2[tour_counter_for_CO2]['tourdistOpt'])/tempCO2[tour_counter_for_CO2]['tourdist'])+" %"
                            except:
                                print "OUPS 1"
                        elif tempCO2[tour_counter_for_CO2]['tourdistOpt'] > 1.25*tempCO2[tour_counter_for_CO2]['tourdist']:
                            
                            edge["group"]+=" DASHED-2"
                            edge["notes"]+="  \n tournée déjà grandement optimisée  \nDistance parcourue pendant le tour: "+str(tempCO2[tour_counter_for_CO2]['tourdist'])+" km  \n"
                            edge["notes"]+="Distance recalculée: "+str(tempCO2[tour_counter_for_CO2]['tourdistOpt'])+" km  \n"
                            try:
                                edge["notes"]+="Taux d'optimisation "+str((tempCO2[tour_counter_for_CO2]['tourdist']-tempCO2[tour_counter_for_CO2]['tourdistOpt'])/tempCO2[tour_counter_for_CO2]['tourdist'])+" %"
                            except:
                                print "OUPS -2"
                        elif tempCO2[tour_counter_for_CO2]['tourdistOpt'] < 1.25*tempCO2[tour_counter_for_CO2]['tourdist'] and tempCO2[tour_counter_for_CO2]['tourdistOpt'] > 1.1*tempCO2[tour_counter_for_CO2]['tourdist']:
                            
                            edge["group"]+=" DASHED-1"
                            edge["notes"]+="  \n  tournée déjà optimisée  \nDistance parcourue pendant le tour: "+str(tempCO2[tour_counter_for_CO2]['tourdist'])+" km  \n"
                            edge["notes"]+="Distance recalculée: "+str(tempCO2[tour_counter_for_CO2]['tourdistOpt'])+" km  \n"
                            try:
                                edge["notes"]+="Taux d'optimisation "+str((tempCO2[tour_counter_for_CO2]['tourdist']-tempCO2[tour_counter_for_CO2]['tourdistOpt'])/tempCO2[tour_counter_for_CO2]['tourdist'])+" %"
                            except:
                                print "OUPS -1"
                        
                        else:

                            edge["notes"]+="  \nDistance parcourue pendant le tour: "+str(tempCO2[tour_counter_for_CO2]['tourdist'])+" km  \n"
                        tour_counter_for_CO2_lock=1
                    except KeyError as e:
                        print e
                else:
                    #print "TOUR NON COMPTE ", tour_indice, "//",tour_counter_for_CO2
                    continue
                    
                    
                    
                my_edges.append({ "data" : edge })
        my_nodes.append({ "data" : node })   
        my_nodesdict[node["id"]]= { "data" : node }
        gig_indice+=1
    #print my_nodes
    #print "<=============================================>"
    #print my_nodesdict
    #print "<=============================================>"
    #print my_edges
     tour_indice +=1
        #print "coords",latitude,"/",longitude
    for key, value in my_nodesdict.iteritems():
            my_nodesdictList.append(value)
    
    #ADDING LIVENATION DETECTOR

    for node in my_nodesdictList:
        for salle in liste_salle_parsee_LN:
            if fuzz.partial_ratio( node['data']['name'], salle)>=80:
                #print "ONE HEERE"
                if round(node['data']['lat'],2)==round(liste_salle_parsee_LN[salle]["lat"],2) and round(node['data']['lng'],2)==round(liste_salle_parsee_LN[salle]["lng"],2) :
                    #print "CONFIRMED LIVENATION!"
                    node['data']['color']="#ff0000"
                    node['data']["notes"]+="  \nLivenation venue"
        #if liste_salle_parsee_LN
        
        
    #ADDING EDGES WEIGHT
    terp= map(lambda x : str(sorted([x["data"]["source"],x["data"]["target"]])),my_edges)
    
    #print terp
    from collections import Counter
    c= Counter()
    for ter in terp:
        c[ter]+=1
    #print c
    for edge in my_edges:
        if str(sorted([edge["data"]["source"],edge["data"]["target"]])) not in terp:
            print "BUG!!!!"
        else:
            if edge["data"]["source"] == edge["data"]["target"]:
                #print c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))]
                edge["data"]["notes"]+="  \ncounted "+str(c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))])+"  times"
                edge["data"]["weight"]= float(1)
                #print "a"
            else:
                edge["data"]["notes"]+="  \ncounted "+str(c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))])+"  times"
                edge["data"]["weight"]= float(c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))])
                #print "b"
    #print my_edges

    
    
    
    

    titredugraphe =artist["name"]+"/BETA_0.91"+"  \nDistance totale parcourue par l'artiste: "+str(tempCO2["totalArtistTourDist"])+" km"
    titredugraphe += "  \nDistance calculee par Concorde: "+str(tempCO2["totalArtistOptTourDist"])+" km"
    if  "OptArtistPercent" in tempCO2 :
         rateOpt=0
         rateOpt=float(tempCO2["OptArtistPercent"])
         titredugraphe+="  \nPourcentage d'optimisation global calcule:"+str(rateOpt)+" %"
         if rateOpt > 10: 
             titredugraphe+="  \nMARGE D'OPTIMISATION IMPORTANTE"
         elif rateOpt < -10:
              titredugraphe+="  \nTOURNEE DEJA OPTIMISEE"
         else: 
             titredugraphe+="  \nGLOBALEMENT IDENTIQUE"
    
    titredugraphe +="  \nPLAYED AT GZ"     
    print      map(lambda x : str(sorted([x["data"]["weight"]])),my_nodesdictList)
    with open(savpath+slugify(artist["name"]+str(artist["mbid"]))+".obj", 'wb') as handle:
                            pickle.dump([titredugraphe,my_nodesdictList,my_edges], handle, protocol=pickle.HIGHEST_PROTOCOL)    
            
    ###CREATE NODE GLOBAL DICT


david-guetta
SKIPPED PROCESSING BUT TAKING INTO ACCOUNT  David Guetta
David Guetta
NOT the-church39-104
NOT on-broadway32-117
NOT vanguard34-118
NOT slide37-122
NOT crobar41-87
NOT the-estate42-71
NOT club-space25-80
NOT velodrome-3318
NOT magazzini-generali459
NOT golden-gate4014
NOT beta39-104
NOT ruby-skye37-122
YES vanguard34-118
1.0
1.0
1.0
1
1
2.0
1.41421356237
NOT spin44-93
YES crobar41-87
1.0
1.0
1.0
1
1
2.0
1.41421356237
NOT glow38-77
NOT pacha-new-york40-73
NOT surfcomber25-80
NOT lovedough-digital54-1
NOT victoria-works53-1
NOT q-club478
NOT ministry-of-sound510
NOT the-best4515
YES pacha-new-york40-73
1.0
1.0
1.0
1
1
2.0
1.41421356237
NOT circa43-79
NOT warehouse-project53-2
NOT stealth-nightclub52-1
YES ministry-of-sound510
1.0
1.0
1.0
1
1
2.0
1.41421356237
NOT nation53-2
NOT ambassadeur5918
NOT siemens-arena5425
NOT lc-club47-1
NOT la-dune431
NOT rockhal-alzette495
NOT four-seasons3031
NOT crobar-w-austin-leeds41-87
NOT fur38-77
NOT fabrik40-3
NOT worthy-farm51-2
NOT los-

7
7
8.0
2.82842712475
NOT led-day-club33-116
YES the-guvernment43-79
1.0
1.0
1.0
1
1
2.0
1.41421356237
NOT capitole-de-quebec46-71
NOT liv25-80
NOT marquee-nyc40-74
NOT story-nightclub25-80
YES encore-beach-club36-115
2.82842712475
2.82842712475
8.0
8
8
9.0
3.0
NOT suburbia-music-festival33-96
YES xs-nightclub36-115
1.0
1.0
1.0
1
1
2.0
1.41421356237
YES new-city-gas45-73
1.0
1.0
1.0
1
1
2.0
1.41421356237
NOT bmo-centre51-114
YES xs-nightclub36-115
1.41421356237
1.41421356237
2.0
2
2
3.0
1.73205080757
YES encore-beach-club36-115
3.0
3.0
9.0
9
9
10.0
3.16227766017
NOT f-me-i-m-famous-pacha381
NOT bigcitybeats-world-club-dome508
NOT world-club-dome508
YES f-me-i-m-famous-pacha381
1.0
1.0
1.0
1
1
2.0
1.41421356237
NOT festival-papillons-de-nuit48-1
YES f-me-i-m-famous-pacha381
1.41421356237
1.41421356237
2.0
2
2
3.0
1.73205080757
NOT national-stadium5221
NOT laugardalshollin64-21
YES f-me-i-m-famous-pacha381
1.73205080757
1.73205080757
3.0
2
3
3.0
1.73205080757
YES f-me-i-m-famous-pacha381

YES xs-nightclub36-115
3.60555127546
3.60555127546
13.0
12
13
13.0
3.60555127546
YES xs-nightclub36-115
3.74165738677
3.74165738677
14.0
14
14
15.0
3.87298334621
YES encore-beach-club36-115
6.2449979984
6.2449979984
39.0
39
39
40.0
6.32455532034
YES xs-nightclub36-115
3.87298334621
3.87298334621
15.0
15
15
16.0
4.0
YES encore-beach-club36-115
6.32455532034
6.32455532034
40.0
40
40
41.0
6.40312423743
NOT pacha-fme-i-m-famous381
NOT gardet5917
YES pacha-fme-i-m-famous381
1.0
1.0
1.0
1
1
2.0
1.41421356237
YES pacha-fme-i-m-famous381
1.41421356237
1.41421356237
2.0
2
2
3.0
1.73205080757
YES pacha-fme-i-m-famous381
1.73205080757
1.73205080757
3.0
2
3
3.0
1.73205080757
YES pacha-fme-i-m-famous381
2.0
2.0
4.0
4
4
5.0
2.2360679775
NOT electrobeach-festival423
YES pacha-fme-i-m-famous381
2.2360679775
2.2360679775
5.0
5
5
6.0
2.44948974278
NOT parookaville516
YES pacha-fme-i-m-famous381
2.44948974278
2.44948974278
6.0
5
6
6.0
2.44948974278
NOT anfiteatro-camerini4511
YES pacha-fme-i-m-famous381


In [20]:
     with open("./my_nodesdictGLOBAL.obj", 'rb') as handle:
          my_nodesdictGLOBAL= pickle.load( handle)

In [6]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

print take(5,my_nodesdictGLOBAL.iteritems())

[('ohio-university39-83', {'data': {'count': 1, 'start': '2016-06-10T18:30:00', 'end': '2016-06-11T18:30:00', 'name': 'ohio-university', 'weight': 1.0, 'lat': 39.3268329, 'lng': -83.0063841, 'notes': '  \n List of the bands that played in this venue:  \nled-zepagain  \n TOTAL: 1 Bands', 'id': 'ohio-university39-83', 'color': '#7f3f3f'}}), ('ohio-university39-82', {'data': {'count': 9, 'start': '2008-04-09T19:00:00', 'end': '2015-11-15T19:00:00', 'name': 'ohio-university', 'weight': 3.3784927944829333, 'lat': 39.324921, 'lng': -82.099709, 'notes': '  \n List of the bands that played in this venue:  \nben-folds  \njosh-ritter  \nmachine-gun-kelly  \nray-wylie-hubbard  \nrayland-baxter  \nthe-blind-boys-of-alabama  \ntyler-farr  \nwiz-khalifa  \nzo  \n TOTAL: 9 Bands', 'id': 'ohio-university39-82', 'color': '#5f7f3f'}}), ('tucson-music-hall32-110', {'data': {'count': 35, 'start': '2007-12-17T00:00:00', 'end': '2017-09-15T19:00:00', 'name': 'tucson-music-hall', 'weight': 6.9258925443875565

In [35]:
# credentials
TOPOGRAM_URL = "http://localhost:3000"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"

In [44]:
# credentials
TOPOGRAM_URL = "https://bandstour.topogram.io"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"

In [23]:
print len(my_nodesdictGLOBAL)

0


In [38]:
# connect to the topogram instance (pass debug=True params for more info )
topogram = TopogramAPIClient(TOPOGRAM_URL) #, debug=True)

# create a new user
try :
    topogram.create_user(USER, PASSWORD)
except ValueError:
    print "> User has already been created."

# login a new user if needed
resp_user_login = topogram.user_login(USER, PASSWORD)
print resp_user_login

assert(resp_user_login["status"] == "success")
assert(resp_user_login["status_code"] == 200)


ERROR:topogram-client:403 Unauthorized request


> User has already been created.
{u'status': u'success', 'status_code': 200, u'data': {u'authToken': u'a2RUJqoz_6IBSmXzWrL-p-dDUuUQT-FAFqjHIoMIblB', u'userId': u'pPd6vpcQSoTuKTrdr'}}


In [24]:
###IMPORT ALL THE BANDS GRAPHS

nodestop=[]
edgestop=[]
tit=[]
import glob, os

print os.getcwd()

for subdir, dirs, files in os.walk(savpath): 
 for file in files:
  if file.endswith('.obj') == True:
    #print file
    with open(os.path.join(subdir, file), 'rb') as handle:
          temp= pickle.load( handle)
    #print temp 
    tit=temp[0].encode("utf-8")
    edgestop=temp[2]
    nodestop=temp[1]
    if (len(nodestop)>50):

      print len(nodestop),len(edgestop) 
      while True:
        try:
    #        print tit
    
            create_topogram(tit, nodestop, edgestop)
            
        except TypeError:
            continue
        
        
        break
print 'DONE'

/home/goonieb/gitrep/bandstour/5_visualizer/topogram-api-clientFOR_TOPO_FROM_BANDSTOUR
123 130
Creating topogram 'Chris Cohen/BETA_0.9  
Distance totale parcourue par l'artiste: 92407.5483559 km  
Distance calculee par Concorde: 74292.8211586 km  
Pourcentage d'optimisation global calcule:19.6030816959 %  
MARGE D'OPTIMISATION IMPORTANTE  
PLAYED AT GZ'


NameError: global name 'topogram' is not defined

In [39]:
###IMPORT ALL THE BANDS GRAPHS

nodestop=[]
edgestop=[]
tit=[]
import glob, os

print os.getcwd()

for subdir, dirs, files in os.walk(savpath): 
 for file in files:
  if file.endswith('david-guetta302bd7b9-d012-4360-897a-93b00c855680.obj') == True:
    #print file
    with open(os.path.join(subdir, file), 'rb') as handle:
          temp= pickle.load( handle)
    #print temp 
    tit=temp[0].encode("utf-8")
    edgestop=temp[2]
    nodestop=temp[1]
    #print edgestop
    #for edge in edgestop:
    #    assert edge["data
    while True:
        try:
    #        print tit
    
            create_topogram(tit, nodestop, edgestop)
            
        except TypeError:
            continue
        except ValueError:
            pass
        
        break
print 'DONE'
        

ERROR:topogram-client:500 - Error : A topogram with the same name already exists


/home/goonieb/gitrep/bandstour/5_visualizer/topogram-api-clientFOR_TOPO_FROM_BANDSTOUR
Creating topogram 'David Guetta/BETA_0.91  
Distance totale parcourue par l'artiste: 1120156.97974 km  
Distance calculee par Concorde: 540532.320416 km  
Pourcentage d'optimisation global calcule:51.7449491284 %  
MARGE D'OPTIMISATION IMPORTANTE  
PLAYED AT GZ'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u"David Guetta/BETA_0.91  \nDistance totale parcourue par l'artiste: 1120156.97974 km  \nDistance calculee par Concorde: 540532.320416 km  \nPourcentage d'optimisation global calcule:51.7449491284 %  \nMARGE D'OPTIMISATION IMPORTANTE  \nPLAYED AT GZ", u'userId': u'pPd6vpcQSoTuKTrdr', u'_id': u'Dzm9RXjFDaAi8iL33', u'slug': u'david-guettabeta_091-distance-totale-parcourue-par-lartiste-112015697974-km-distance-calculee-par-concorde-540532320416-km-pourcentage-doptimisation-global-calcule517449491284-marge-doptimisation-importante-pla

In [50]:
###IMPORT THE GLOBAL VENUE  GRAPH
my_nodesdictList=[]
nodestopo=[]
edgestopo=[]
with open("./my_nodesdictGLOBAL.obj", 'rb') as handle:
          my_nodesdictGLOBAL= pickle.load( handle)
        
for key, value in my_nodesdictGLOBAL.iteritems():
            my_nodesdictGLOBAL[key]["data"].pop('count')
        
for key, value in my_nodesdictGLOBAL.iteritems():
            my_nodesdictList.append(value)
        
nodestopo=sorted(my_nodesdictList, key= lambda (v) : v['data']['weight'], reverse=True)[:5000]
print len(nodestopo)



title="global venue graph /BETA 0.3"

while True:
        try:
            print nodestopo
            create_topogram(title, nodestopo, edgestopo)
        
        except TypeError:
            continue
        break
        

ERROR:topogram-client:500 - Error : A topogram with the same name already exists
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


5000 nodes deleted
5000 nodes created.


ValueError: No JSON object could be decoded

In [14]:
with open("./bands_dicts/the-real-mckenzies01612542-01ec-4fc4-a9ad-b6a6e33f11d7.obj", 'rb') as handle:
          DEBUG= pickle.load( handle)


In [18]:
print DEBUG[:2]


[u"The Real McKenzies/BETA_0.8  \nDistance totale parcourue par l'artiste: 180691.650535 km  \nDistance calculee par Concorde: 156448.795249 km  \nPourcentage d'optimisation global calculee:13.4166992303 %  \nMARGE D'OPTIMISATION IMPORTANTE", [{'data': {'end': '2015-03-27T20:30:00', 'name': 'the-horseshoe-tavern', 'weight': 1.0, 'color': '#7f663f', 'notes': 'name : the-horseshoe-tavern  \nlat : 43.6490438  \nlng :-79.395913  \n weight :1  \nstart : 2015-03-26T20:30:00  \nend : 2015-03-27T20:30:00  \ncolor :#7f663f  \n  <a href="https://google.com/search?q=the-horseshoe-tavern" target="_blank">the-horseshoe-tavern</a>', 'start': '2015-03-26T20:30:00', 'lat': 43.6490438, 'lng': -79.395913, 'id': 'the-horseshoe-tavern43-79'}}, {'data': {'end': '2012-05-09T20:00:00', 'name': 'the-pyramid-cabaret', 'weight': 1.0, 'color': '#723f7f', 'notes': 'name : the-pyramid-cabaret  \nlat : 49.890735  \nlng :-97.137711  \n weight :1  \nstart : 2012-05-08T20:00:00  \nend : 2012-05-09T20:00:00  \ncolor :#